# Introduction

This project focuses on applying Multi-Criteria Decision Making (MCDM) techniques to a dataset of Spotify songs. Our objective is to determine the relative importance of various criteria by employing both subjective and objective weighting methods. In this study, we use the Analytic Hierarchy Process (AHP) as the subjective approach and the Entropy method as the objective approach.

After determining the weights, we apply five different MCDM methods – Weighted Sum, Weighted Product, WASPAS, TOPSIS, and PROMETHEE – to rank the alternatives. This notebook documents the complete process, from data preprocessing to the interpretation of results, and provides a comparative analysis of the applied techniques.


# Data Description and Preprocessing

The dataset contains detailed information on Spotify songs, with each song characterized by various criteria such as acoustic features, popularity metrics, and other relevant attributes. This section outlines the following key steps:

- **Data Cleaning:** Handling missing values, correcting inconsistencies, and ensuring overall data quality.
- **Normalization:** Scaling the data appropriately to facilitate fair comparisons across different criteria.
- **Criteria Selection:** Identifying and selecting the most influential attributes for our MCDM analysis.

These steps are essential to ensure that the subsequent analysis is performed on a robust and reliable dataset.


In [1]:
import pandas as pd
import numpy as np
import copy

data = pd.read_csv('spotify-2023.csv', encoding='latin1')

In [2]:
data

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


In [3]:
data.isna().sum()

,0
track_name,0
artist(s)_name,0
artist_count,0
released_year,0
released_month,0
released_day,0
in_spotify_playlists,0
in_spotify_charts,0
streams,0
in_apple_playlists,0


In this study, we focus on seven key criteria: `in_spotify_playlists`, `in_spotify_charts`, `streams`, `in_apple_playlists`, `in_apple_charts`, `in_deezer_playlists`, and `in_shazam_charts`. These criteria were selected based on their relevance in assessing a song's popularity and visibility across major music platforms.


The `in_shazam_charts` column contains 50 missing values out of a total of 953 rows, accounting for approximately 5.2% of the dataset. Since this percentage is slightly above the commonly used 5% threshold for handling missing data, one possible approach is to remove these rows to maintain data integrity.

In [4]:
data.dropna(subset=['in_shazam_charts'], inplace=True)

In [5]:
data.isna().sum()

,0
track_name,0
artist(s)_name,0
artist_count,0
released_year,0
released_month,0
released_day,0
in_spotify_playlists,0
in_spotify_charts,0
streams,0
in_apple_playlists,0


* To focus on the oldest 200 songs in the dataset, we will first sort the data by the release date in ascending order.

In [6]:
data_sorted = data.sort_values(by=['released_year', 'released_month', 'released_day'], ascending=[True, True, True])

In [7]:
spotify_data = data_sorted[['track_name','artist(s)_name','in_spotify_playlists','in_spotify_charts','streams','in_apple_playlists','in_apple_charts','in_deezer_playlists','in_shazam_charts']]

In [8]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 903 entries, 439 to 68
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            903 non-null    object
 1   artist(s)_name        903 non-null    object
 2   in_spotify_playlists  903 non-null    int64 
 3   in_spotify_charts     903 non-null    int64 
 4   streams               903 non-null    object
 5   in_apple_playlists    903 non-null    int64 
 6   in_apple_charts       903 non-null    int64 
 7   in_deezer_playlists   903 non-null    object
 8   in_shazam_charts      903 non-null    object
dtypes: int64(4), object(5)
memory usage: 70.5+ KB


In [9]:
def print_invalid_rows(df, columns):
    """
    Prints rows where specified columns contain non-numeric values.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to check.
    """
    for col in columns:
        invalid_rows = df[~df[col].astype(str).str.replace(',', '').str.match(r'^\d+(\.\d+)?$', na=False)]

        if not invalid_rows.empty:
            print(f"Invalid values in column '{col}':")
            print(invalid_rows)
            print("\n" + "-" * 50 + "\n")

columns_to_check = ['streams', 'in_deezer_playlists', 'in_shazam_charts']
print_invalid_rows(spotify_data, columns_to_check)


Invalid values in column 'streams':
                              track_name     artist(s)_name  \
574  Love Grows (Where My Rosemary Goes)  Edison Lighthouse   

     in_spotify_playlists  in_spotify_charts  \
574                  2877                  0   

                                               streams  in_apple_playlists  \
574  BPM110KeyAModeMajorDanceability53Valence75Ener...                  16   

     in_apple_charts in_deezer_playlists in_shazam_charts  
574                0                  54                0  

--------------------------------------------------



* The `streams` column contains invalid data in row 574, where the expected
numerical value is replaced with a concatenated string of multiple attributes. This issue likely resulted from a data formatting or parsing error and should be corrected or removed to ensure accurate analysis.


In [10]:
def filter_numeric_rows(df, columns):
    """
    Filters a DataFrame to keep only rows where specified columns contain valid numeric values.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to check.

    Returns:
        pd.DataFrame: Filtered DataFrame with only numeric values in specified columns.
    """
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, setting errors to NaN

    return df.dropna(subset=columns)  # Drop rows where any of the specified columns have NaN


columns_to_check = ['streams', 'in_deezer_playlists', 'in_shazam_charts']
spotify_data_cleaned = filter_numeric_rows(spotify_data, columns_to_check)

print(spotify_data_cleaned.dtypes)
print(spotify_data_cleaned.isna().sum())

track_name               object
artist(s)_name           object
in_spotify_playlists      int64
in_spotify_charts         int64
streams                 float64
in_apple_playlists        int64
in_apple_charts           int64
in_deezer_playlists     float64
in_shazam_charts        float64
dtype: object
track_name              0
artist(s)_name          0
in_spotify_playlists    0
in_spotify_charts       0
streams                 0
in_apple_playlists      0
in_apple_charts         0
in_deezer_playlists     0
in_shazam_charts        0
dtype: int64


<ipython-input-10-9429e8bdbdec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, setting errors to NaN


In [11]:
spotify_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 829 entries, 439 to 68
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            829 non-null    object 
 1   artist(s)_name        829 non-null    object 
 2   in_spotify_playlists  829 non-null    int64  
 3   in_spotify_charts     829 non-null    int64  
 4   streams               829 non-null    float64
 5   in_apple_playlists    829 non-null    int64  
 6   in_apple_charts       829 non-null    int64  
 7   in_deezer_playlists   829 non-null    float64
 8   in_shazam_charts      829 non-null    float64
dtypes: float64(3), int64(4), object(2)
memory usage: 64.8+ KB


* After sorting, we will select the top 200 entries, ensuring that all further analyses and MCDM applications are performed exclusively on these songs.

In [12]:
oldest_200_songs = spotify_data_cleaned.head(200)

In [13]:
oldest_200_songs

,track_name,artist(s)_name,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_shazam_charts
439,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",323,0,90598517.0,4,0,14.0,0.0
469,White Christmas,"Bing Crosby, John Scott Trotter & His Orchestr...",11940,0,395591396.0,73,79,123.0,0.0
460,The Christmas Song (Merry Christmas To You) - ...,Nat King Cole,11500,0,389771964.0,140,72,251.0,0.0
466,Let It Snow! Let It Snow! Let It Snow!,"Frank Sinatra, B. Swanson Quartet",10585,0,473248298.0,126,108,406.0,0.0
459,A Holly Jolly Christmas - Single Version,Burl Ives,7930,0,395591396.0,108,120,73.0,0.0
...,...,...,...,...,...,...,...,...,...
825,Flowers,Lauren Spencer Smith,801,0,184826429.0,42,9,24.0,1.0
612,Every Summertime,NIKI,1211,2,290228626.0,30,2,5.0,6.0
642,Se Le Ve,"Arcangel, De La Ghetto, Justin Quiles, Lenny T...",1560,0,223319934.0,72,0,0.0,0.0
605,OUT OUT (feat. Charli XCX & Saweetie),"Charli XCX, Jax Jones, Joel Corry, Saweetie",6890,0,427486004.0,122,11,201.0,1.0


In [14]:
oldest_200_songs.describe()

,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_shazam_charts
count,200.000000,200.000000,2.000000e+02,200.000000,200.00000,200.000000,200.000000
mean,6832.275000,9.505000,7.376846e+08,83.290000,52.76000,242.315000,40.735000
std,5445.192724,16.658557,4.754806e+08,79.673491,48.18716,247.813931,101.638164
min,31.000000,0.000000,3.841196e+07,0.000000,0.00000,0.000000,0.000000
25%,2818.500000,0.000000,4.044830e+08,24.000000,8.75000,52.000000,0.000000
50%,5386.500000,2.000000,6.380300e+08,65.000000,45.00000,141.500000,2.000000
75%,9426.500000,14.000000,9.849468e+08,120.250000,86.25000,369.750000,25.500000
max,29499.000000,110.000000,2.808097e+09,492.000000,266.00000,974.000000,734.000000


In [15]:
criteria_df = oldest_200_songs.iloc[:, 2:]
matrix = criteria_df.to_numpy()

In [16]:
matrix

array([[3.23000000e+02, 0.00000000e+00, 9.05985170e+07, ...,
        0.00000000e+00, 1.40000000e+01, 0.00000000e+00],
       [1.19400000e+04, 0.00000000e+00, 3.95591396e+08, ...,
        7.90000000e+01, 1.23000000e+02, 0.00000000e+00],
       [1.15000000e+04, 0.00000000e+00, 3.89771964e+08, ...,
        7.20000000e+01, 2.51000000e+02, 0.00000000e+00],
       ...,
       [1.56000000e+03, 0.00000000e+00, 2.23319934e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.89000000e+03, 0.00000000e+00, 4.27486004e+08, ...,
        1.10000000e+01, 2.01000000e+02, 1.00000000e+00],
       [1.24030000e+04, 0.00000000e+00, 6.74772936e+08, ...,
        6.30000000e+01, 4.65000000e+02, 1.10000000e+01]])

# Weight Determination Methods – Entropy Method

The Entropy method offers an objective approach by calculating weights based on the inherent information present in the dataset. The key steps are:

- **Normalization:** Standardizing the data to remove scale effects across criteria.
- **Entropy Calculation:** Measuring the degree of variation or dispersion within each criterion.
- **Weight Derivation:** Assigning weights inversely proportional to the entropy values; criteria with higher variation are given greater importance.

This method complements AHP by providing data-driven insights that balance the subjective judgments with objective measures.


1. **Normalization:**
$$
p_{ij} = \frac{x_{ij}}{\sum_{i=1}^{n} x_{ij}}
$$
where \(x_{ij}\) is the performance value for alternative \(i\) under criterion \(j\).

In [17]:
def normalize_matrix(matrix):
    col_sums = np.sum(matrix, axis=0)
    col_sums[col_sums == 0] = 1  # Avoid division by zero
    return matrix / col_sums

normalized_matrix = normalize_matrix(matrix)

In [18]:
normalized_matrix

array([[0.00023638, 0.        , 0.00061407, ..., 0.        , 0.00028888,
        0.        ],
       [0.00873794, 0.        , 0.0026813 , ..., 0.00748673, 0.00253802,
        0.        ],
       [0.00841594, 0.        , 0.00264186, ..., 0.00682335, 0.00517921,
        0.        ],
       ...,
       [0.00114164, 0.        , 0.00151365, ..., 0.        , 0.        ,
        0.        ],
       [0.00504224, 0.        , 0.00289749, ..., 0.00104246, 0.00414749,
        0.00012274],
       [0.00907677, 0.        , 0.00457359, ..., 0.00597043, 0.00959495,
        0.00135019]])

In [19]:
def compute_pij_ln_pij(matrix):
    return np.where(matrix > 0, matrix * np.log(matrix), 0)

pij_ln_pij = compute_pij_ln_pij(normalized_matrix)

<ipython-input-19-ab4c75007a0d>:2: RuntimeWarning: divide by zero encountered in log
  return np.where(matrix > 0, matrix * np.log(matrix), 0)
<ipython-input-19-ab4c75007a0d>:2: RuntimeWarning: invalid value encountered in multiply
  return np.where(matrix > 0, matrix * np.log(matrix), 0)


In [20]:
pij_ln_pij

array([[-0.00197378,  0.        , -0.00454132, ...,  0.        ,
        -0.00235423,  0.        ],
       [-0.04141854,  0.        , -0.01587722, ..., -0.03664473,
        -0.01516814,  0.        ],
       [-0.04020822,  0.        , -0.0156828 , ..., -0.03403081,
        -0.02725871,  0.        ],
       ...,
       [-0.00773494,  0.        , -0.00982851, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02667299,  0.        , -0.01693265, ..., -0.00715769,
        -0.02275005, -0.00110536],
       [-0.04267931,  0.        , -0.02464001, ..., -0.0305742 ,
        -0.04458311, -0.0089214 ]])

2. **Entropy Calculation:**
$$
E_j = -k \sum_{i=1}^{n} p_{ij} \ln(p_{ij}), \quad \text{with } k = \frac{1}{\ln(n)}
$$


In [21]:
n = normalized_matrix.shape[0]  # Number of alternatives
k = 1 / np.log(n)
ej_matrix = -k * pij_ln_pij.sum(axis=0)
ej_matrix

array([0.94513165, 0.79484053, 0.96370182, 0.92232004, 0.91417528,
       0.907782  , 0.71018386])

3. **Weight Derivation:**
$$
w_j = \frac{1 - E_j}{\sum_{j=1}^{m} (1 - E_j)}
$$

In [22]:
def compute_wj_matrix(ej):
    one_minus_ej = 1 - ej
    return one_minus_ej / one_minus_ej.sum()

wj_matrix = compute_wj_matrix(ej_matrix)
wj_matrix

array([0.06517477, 0.24369645, 0.0431164 , 0.0922713 , 0.10194597,
       0.10954015, 0.34425497])

In [23]:
criteria_names = criteria_df.columns.tolist()
for name, weight in zip(criteria_names, wj_matrix):
    print(f"{name}: {weight:.4f}")

in_spotify_playlists: 0.0652
in_spotify_charts: 0.2437
streams: 0.0431
in_apple_playlists: 0.0923
in_apple_charts: 0.1019
in_deezer_playlists: 0.1095
in_shazam_charts: 0.3443


# Weight Determination Methods – AHP

The Analytic Hierarchy Process (AHP) is used as a subjective method to determine the weights of the criteria. The main steps include:

- **Pairwise Comparison Matrix:** Constructing a matrix where each criterion is compared against every other criterion based on expert judgment.
- **Priority Vector Calculation:** Extracting the relative weights from the pairwise comparison matrix using eigenvalue methods.
- **Consistency Check:** Verifying the consistency of the comparisons to ensure that the judgments are reliable.

AHP provides a structured framework that emphasizes the decision makers’ preferences and highlights the relative importance of each criterion.

In our analysis, we focus on seven key criteria that capture various aspects of song popularity and platform presence. These criteria are:

1. **in_spotify_playlists**: Frequency or inclusion of songs in Spotify playlists.
2. **in_spotify_charts**: Representation of songs in Spotify charts.
3. **streams**: Total number of streams a song has received.
4. **in_apple_playlists**: Frequency or inclusion of songs in Apple playlists.
5. **in_apple_charts**: Representation of songs in Apple charts.
6. **in_deezer_playlists**: Frequency or inclusion of songs in Deezer playlists.
7. **in_shazam_charts**: Representation of songs in Shazam charts.

### AHP Scale for Pairwise Comparisons

To derive the relative importance of each criterion, we use the Analytic Hierarchy Process (AHP) scale. The scale assigns the following values:

- **1**: Equal importance
- **3**: Moderate importance
- **5**: Strong importance
- **7**: Very strong importance
- **9**: Extreme importance
- **1/3, 1/5, 1/7, 1/9**: Inverse values used when one criterion is less important than the other

This structured scale ensures a consistent and systematic approach to evaluate the criteria, ultimately helping to determine accurate weights for our MCDM analysis.


In [24]:
matrix_ahp = np.array([
    [1.00, 3.00, 1/5, 2.00, 3.00, 1/3, 5.00],
    [1/3, 1.00, 1/7, 1/2, 1.00, 1/5, 3.00],
    [5.00, 7.00, 1.00, 6.00, 7.00, 4.00, 9.00],
    [1/2, 2.00, 1/6, 1.00, 2.00, 1/3, 4.00],
    [1/3, 1.00, 1/7, 1/2, 1.00, 1/5, 3.00],
    [3.00, 5.00, 1/4, 3.00, 5.00, 1.00, 7.00],
    [1/5, 1/3, 1/9, 1/4, 1/3, 1/7, 1.00]
])

labels = [ "in_spotify_playlists",
    "in_spotify_charts", "streams", "in_apple_playlists",
    "in_apple_charts", "in_deezer_playlists", "in_shazam_charts"
]

df = pd.DataFrame(matrix_ahp, columns=labels, index=labels)
df

,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_shazam_charts
in_spotify_playlists,1.000000,3.000000,0.200000,2.00,3.000000,0.333333,5.0
in_spotify_charts,0.333333,1.000000,0.142857,0.50,1.000000,0.200000,3.0
streams,5.000000,7.000000,1.000000,6.00,7.000000,4.000000,9.0
in_apple_playlists,0.500000,2.000000,0.166667,1.00,2.000000,0.333333,4.0
in_apple_charts,0.333333,1.000000,0.142857,0.50,1.000000,0.200000,3.0
in_deezer_playlists,3.000000,5.000000,0.250000,3.00,5.000000,1.000000,7.0
in_shazam_charts,0.200000,0.333333,0.111111,0.25,0.333333,0.142857,1.0


In [25]:
matrix_ahp

array([[1.        , 3.        , 0.2       , 2.        , 3.        ,
        0.33333333, 5.        ],
       [0.33333333, 1.        , 0.14285714, 0.5       , 1.        ,
        0.2       , 3.        ],
       [5.        , 7.        , 1.        , 6.        , 7.        ,
        4.        , 9.        ],
       [0.5       , 2.        , 0.16666667, 1.        , 2.        ,
        0.33333333, 4.        ],
       [0.33333333, 1.        , 0.14285714, 0.5       , 1.        ,
        0.2       , 3.        ],
       [3.        , 5.        , 0.25      , 3.        , 5.        ,
        1.        , 7.        ],
       [0.2       , 0.33333333, 0.11111111, 0.25      , 0.33333333,
        0.14285714, 1.        ]])

1. **Normalization of the Matrix:**
   - The function `normalize_matrix` divides each element \( a_{ij} \) of the matrix by the sum of its corresponding column.
   - **Formula:**
$$
n_{ij} = \frac{a_{ij}}{\sum_{i=1}^{m} a_{ij}}
$$
     where:
     - \( a_{ij} \) is the element in the \( i \)-th row and \( j \)-th column of the original matrix.
     - \( n_{ij} \) is the normalized value.
     - \( m \) is the number of rows.


In [26]:
def normalize_matrix(matrix):
    column_sums = matrix.sum(axis=0)
    return matrix / column_sums

# Normalize the matrix
normalized_matrix = normalize_matrix(matrix_ahp)
print("\nNormalized Matrix:\n", np.round(normalized_matrix, 3))


Normalized Matrix:
 [[0.096 0.155 0.099 0.151 0.155 0.054 0.156]
 [0.032 0.052 0.071 0.038 0.052 0.032 0.094]
 [0.482 0.362 0.497 0.453 0.362 0.644 0.281]
 [0.048 0.103 0.083 0.075 0.103 0.054 0.125]
 [0.032 0.052 0.071 0.038 0.052 0.032 0.094]
 [0.289 0.259 0.124 0.226 0.259 0.161 0.219]
 [0.019 0.017 0.055 0.019 0.017 0.023 0.031]]


2. **Calculation of Criteria Weights:**
   - The function `calculate_weights` computes the weight for each criterion by taking the mean of the normalized values across the row.
   - **Formula:**
$$
w_i = \frac{1}{n} \sum_{j=1}^{n} n_{ij}
$$
     where:
     - \( w_i \) is the weight of the \( i \)-th criterion.
     - \( n \) is the number of columns (criteria).

In [27]:
def calculate_weights(normalized_matrix):
    return normalized_matrix.mean(axis=1)

# Calculate weights
weights_ahp = calculate_weights(normalized_matrix)
print("\nCriteria Weights:", np.round(weights_ahp, 3))


Criteria Weights: [0.124 0.053 0.44  0.085 0.053 0.22  0.026]


- **Consistency Check:** Compute the Consistency Index (CI) and Consistency Ratio (CR):
$$
\text{CI} = \frac{\lambda_{\text{max}} - n}{n - 1}, \quad \text{CR} = \frac{\text{CI}}{\text{RI}}
$$
where \(n\) is the number of criteria and RI is the Random Index.

* RI_dict = {1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.
41, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.48, 13: 1.56, 14: 1.57, 15: 1.59}

In [28]:
def ahp_consistency_check(matrix, weights):
    n = matrix.shape[0]
    AW = matrix @ weights
    lambda_max = np.mean(AW / weights)
    CI = (lambda_max - n) / (n - 1)
    RI = 1.32  # Random Index for n=7
    CR = CI / RI
    return CR

CR = ahp_consistency_check(matrix_ahp, weights_ahp)
print(f"\nConsistency Ratio (CR): {CR:.3f}")
if CR < 0.1:
    print("Consistent pairwise comparisons!")
else:
    print("Inconsistent! Revise your comparisons.")


Consistency Ratio (CR): 0.039
Consistent pairwise comparisons!


# Weighted Sum Method (WSM)

The Weighted Sum Method is one of the simplest MCDM techniques and involves the following steps:

- Multiplying each criterion value by its corresponding weight.
- Summing these weighted values for each alternative.
- Ranking the alternatives based on their total scores.

This method is particularly effective when the criteria are positively correlated with the performance of alternatives.


In [29]:
matrix

array([[3.23000000e+02, 0.00000000e+00, 9.05985170e+07, ...,
        0.00000000e+00, 1.40000000e+01, 0.00000000e+00],
       [1.19400000e+04, 0.00000000e+00, 3.95591396e+08, ...,
        7.90000000e+01, 1.23000000e+02, 0.00000000e+00],
       [1.15000000e+04, 0.00000000e+00, 3.89771964e+08, ...,
        7.20000000e+01, 2.51000000e+02, 0.00000000e+00],
       ...,
       [1.56000000e+03, 0.00000000e+00, 2.23319934e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.89000000e+03, 0.00000000e+00, 4.27486004e+08, ...,
        1.10000000e+01, 2.01000000e+02, 1.00000000e+00],
       [1.24030000e+04, 0.00000000e+00, 6.74772936e+08, ...,
        6.30000000e+01, 4.65000000e+02, 1.10000000e+01]])

### Criteria Type and Reason

| **Criterion**             | **Type**         | **Reason**                                                                 |
|---------------------------|------------------|----------------------------------------------------------------------------|
| **in_spotify_playlists**   | Maximization     | More playlists indicate higher popularity.                                 |
| **in_spotify_charts**      | Minimization     | Lower rank numbers (e.g., rank 1) indicate better performance.                        |
| **streams**                | Maximization     | More streams indicate higher popularity and engagement..                    |
| **in_apple_playlists**     | Maximization     | More playlists indicate higher popularity.                                 |
| **in_apple_charts**        | Minimization     | Lower rank numbers (e.g., rank 1) indicate better performance.                        |
| **in_deezer_playlists**    | Maximization     | More playlists indicate higher popularity.                                 |
| **in_shazam_charts**       | Minimization     | Lower rank numbers (e.g., rank 1) indicate better performance.
   |

### Handling NaN Values in Normalized Matrix

During the normalization process, I encountered **NaN (Not a Number)** values in the normalized matrix. This occurred because:

1. **Division by Zero**:
   - For **minimization criteria**, the normalization formula is \( \text{min}(x) / x \).
   - If the **minimum value** of a column is **0**, and there are **zero values** in the column, the division \( 0 / 0 \) results in **NaN**.

2. **Impact of Zero Values**:
   - Columns like `in_spotify_charts`, `in_apple_charts`, and `in_shazam_charts` contain **zero values**, which caused the **NaN values** in the normalized matrix.

3. **Solution**:
   - To handle this issue, I replaced the **minimum value** of **0** with a **small non-zero value** (e.g., \( 1 \times 10^{-10} \)) for columns where the minimum value is **0**.
   - This ensures that division by zero is avoided, and the normalized matrix does not contain **NaN values**.

4. **Result**:
   - After applying this fix, the normalized matrix no longer contains **NaN values**, and all values are properly scaled between **0** and **1**.

In [30]:
criteria_types = ["Max", "Min", "Max", "Max", "Min", "Max", "Min"]

def normalize_matrix(matrix, criteria_types):
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop over columns
        column = matrix[:, j]

        if criteria_types[j] == "Max":
            # Max normalization: x / max(x)
            max_value = np.max(column)
            normalized_matrix[:, j] = column / max_value

        else:  # Min criteria
            # Min normalization: min(x) / x
            min_value = np.min(column)
            if min_value == 0:  # Handle division by zero
                min_value = 1e-10  # Replace 0 with a small non-zero value
            normalized_matrix[:, j] = min_value / np.where(column == 0, 1e-10, column)

    return normalized_matrix

normalized_ws_matrix = normalize_matrix(matrix, criteria_types)

print("Normalized Matrix:")
print(normalized_ws_matrix)

Normalized Matrix:
[[1.09495237e-02 1.00000000e+00 3.22633198e-02 ... 1.00000000e+00
  1.43737166e-02 1.00000000e+00]
 [4.04759483e-01 1.00000000e+00 1.40875283e-01 ... 1.26582278e-12
  1.26283368e-01 1.00000000e+00]
 [3.89843724e-01 1.00000000e+00 1.38802907e-01 ... 1.38888889e-12
  2.57700205e-01 1.00000000e+00]
 ...
 [5.28831486e-02 1.00000000e+00 7.95271566e-02 ... 1.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.33567240e-01 1.00000000e+00 1.52233371e-01 ... 9.09090909e-12
  2.06365503e-01 1.00000000e-10]
 [4.20454931e-01 1.00000000e+00 2.40295490e-01 ... 1.58730159e-12
  4.77412731e-01 9.09090909e-12]]


**Formula:**

$$
\text{Score}_i = \sum_{j=1}^{m} w_j \cdot x_{ij}
$$

where:
- \(x_{ij}\) is the performance of alternative \(i\) under criterion \(j\).
- \(w_j\) is the weight for criterion \(j\).

In [31]:
def weighted_sum_matrix(normalized_matrix, weights):
    return np.dot(normalized_matrix, weights)

# using entropy weights
weighted_sum_entweight = weighted_sum_matrix(normalized_ws_matrix, wj_matrix)

# using ahp weights
weighted_sum_ahpweight = weighted_sum_matrix(normalized_ws_matrix, weights_ahp)

In [32]:
# Create a DataFrame for ranking
ranking_df = pd.DataFrame({
    "Track Name": oldest_200_songs["track_name"],  # Song name column
    "Artist Name": oldest_200_songs["artist(s)_name"],  # Artist name column
    "Entropy Weighted Score": weighted_sum_entweight,  # Entropy weighted scores
    "AHP Weighted Score": weighted_sum_ahpweight  # AHP weighted scores
})

# Sort by Entropy Weighted Score (descending order)
ranking_df_entropy = ranking_df.sort_values(by="Entropy Weighted Score", ascending=False)
ranking_df_entropy["Entropy Rank"] = range(1, len(ranking_df_entropy) + 1)

# Sort by AHP Weighted Score (descending order)
ranking_df_ahp = ranking_df.sort_values(by="AHP Weighted Score", ascending=False)
ranking_df_ahp["AHP Rank"] = range(1, len(ranking_df_ahp) + 1)

In [33]:
print("Ranking List Using Entropy Weights:")
ranking_df_entropy[["Track Name", "Artist Name", "Entropy Weighted Score", "Entropy Rank"]].head(10)

Ranking List Using Entropy Weights:


,Track Name,Artist Name,Entropy Weighted Score,Entropy Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.797896,1
510,Infinity,Jaymes Young,0.762236,2
424,Running Up That Hill (A Deal With God),Kate Bush,0.749845,3
452,Holly Jolly Christmas,Michael Bublï¿,0.747787,4
721,"jealousy, jealousy",Olivia Rodrigo,0.717064,5
462,Do They Know It's Christmas? - 1984 Version,Band Aid,0.716345,6
607,2055,Sleepy hallow,0.714792,7
840,Pass The Dutchie,Musical Youth,0.714076,8
642,Se Le Ve,"Arcangel, De La Ghetto, Justin Quiles, Lenny T...",0.710276,9
470,Driving Home for Christmas - 2019 Remaster,Chris Rea,0.708506,10


In [34]:
ranking_df_entropy.to_excel('ranking_WS_entropy.xlsx', index=False)

In [35]:
print("\nRanking List Using AHP Weights:")
ranking_df_ahp[["Track Name", "Artist Name", "AHP Weighted Score", "AHP Rank"]].head(10)


Ranking List Using AHP Weights:


,Track Name,Artist Name,AHP Weighted Score,AHP Rank
41,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",0.795348,1
84,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",0.779891,2
621,Lucid Dreams,Juice WRLD,0.665969,3
187,Circles,Post Malone,0.626742,4
166,Every Breath You Take - Remastered 2003,The Police,0.589675,5
526,Beggin,Mï¿½ï¿½ne,0.552022,6
516,Kiss Me More (feat. SZA),"SZA, Doja Cat",0.544192,7
169,When I Was Your Man,Bruno Mars,0.531170,8
167,The Night We Met,Lord Huron,0.522499,9
170,Let Me Down Slowly,Alec Benjamin,0.518650,10


In [36]:
ranking_df_ahp.to_excel('ranking_WS_ahp.xlsx', index=False)

# Weighted Product Method (WPM)

The Weighted Product Method operates on a multiplicative basis rather than an additive one. Its main steps are:

- Multiplying the criteria values, each raised to the power of its corresponding weight.
- Comparing the resulting products to rank the alternatives.

WPM is advantageous when dealing with ratios or relative scales, as it tends to reduce the influence of extreme values.


**Formula:**

$$
\text{Score}_i = \prod_{j=1}^{m} \left( x_{ij} \right)^{w_j}
$$

In [37]:
def weighted_product_matrix(normalized_matrix, weights):
    return np.prod(np.power(normalized_matrix, weights), axis=1)

# using entropy weights
weighted_prod_entweight = weighted_product_matrix(normalized_ws_matrix, wj_matrix)

# using ahp weights
weighted_prod_ahpweight = weighted_product_matrix(normalized_ws_matrix, weights_ahp)

In [38]:
# Create a DataFrame for ranking
ranking_df_prod = pd.DataFrame({
    "Track Name": oldest_200_songs["track_name"],  # Song name column
    "Artist Name": oldest_200_songs["artist(s)_name"],  # Artist name column
    "Entropy Weighted Score": weighted_prod_entweight,  # Entropy weighted scores
    "AHP Weighted Score": weighted_prod_ahpweight  # AHP weighted scores
})

# Sort by Entropy Weighted Score (descending order)
ranking_df_entropy_prod = ranking_df_prod.sort_values(by="Entropy Weighted Score", ascending=False)
ranking_df_entropy_prod["Entropy Rank"] = range(1, len(ranking_df_entropy_prod) + 1)

# Sort by AHP Weighted Score (descending order)
ranking_df_ahp_prod = ranking_df_prod.sort_values(by="AHP Weighted Score", ascending=False)
ranking_df_ahp_prod["AHP Rank"] = range(1, len(ranking_df_ahp_prod) + 1)

In [39]:
print("Ranking List Using Entropy Weights:")
ranking_df_entropy_prod[["Track Name", "Artist Name", "Entropy Weighted Score", "Entropy Rank"]].head(10)

Ranking List Using Entropy Weights:


,Track Name,Artist Name,Entropy Weighted Score,Entropy Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.658921,1
510,Infinity,Jaymes Young,0.576198,2
452,Holly Jolly Christmas,Michael Bublï¿,0.486658,3
607,2055,Sleepy hallow,0.434419,4
721,"jealousy, jealousy",Olivia Rodrigo,0.425895,5
609,Happier Than Ever - Edit,Billie Eilish,0.400517,6
470,Driving Home for Christmas - 2019 Remaster,Chris Rea,0.340354,7
461,Wonderful Christmastime - Edited Version / Rem...,Paul McCartney,0.293156,8
439,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",0.258977,9
424,Running Up That Hill (A Deal With God),Kate Bush,0.075542,10


In [40]:
ranking_df_entropy_prod.to_excel('ranking_WP_entropy.xlsx', index=False)

In [41]:
print("\nRanking List Using AHP Weights:")
ranking_df_ahp_prod[["Track Name", "Artist Name", "AHP Weighted Score", "AHP Rank"]].head(10)


Ranking List Using AHP Weights:


,Track Name,Artist Name,AHP Weighted Score,AHP Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.365738,1
510,Infinity,Jaymes Young,0.302333,2
452,Holly Jolly Christmas,Michael Bublï¿,0.201587,3
721,"jealousy, jealousy",Olivia Rodrigo,0.162979,4
424,Running Up That Hill (A Deal With God),Kate Bush,0.149470,5
607,2055,Sleepy hallow,0.149370,6
609,Happier Than Ever - Edit,Billie Eilish,0.114479,7
613,Talking To The Moon,Bruno Mars,0.102962,8
621,Lucid Dreams,Juice WRLD,0.093304,9
911,Sparks,Coldplay,0.093041,10


In [42]:
ranking_df_ahp_prod.to_excel('ranking_WP_ahp.xlsx', index=False)

# WASPAS Method

The WASPAS method combines the strengths of both the Weighted Sum and Weighted Product methods. Its process involves:

- Calculating the additive aggregation (as in WSM) and the multiplicative aggregation (as in WPM) of the criteria.
- Merging these two scores using a predefined coefficient to arrive at a final ranking.

This hybrid approach aims to harness the benefits of both methods, leading to a more balanced decision-making process.


**Formula:**

WASPAS combines the Weighted Sum and Weighted Product methods:
$$
Q_i = \lambda \cdot \text{WSM}_i + (1 - \lambda) \cdot \text{WPM}_i
$$
where typically \(\lambda = 0.5\).

In [43]:
lambda_vector = np.full(200, 0.5)

#using entropy weights
waspas_scores_ent = lambda_vector * weighted_sum_entweight + (1 - lambda_vector) * weighted_prod_entweight

#using ahp weights
waspas_scores_ahp = lambda_vector * weighted_sum_ahpweight + (1 - lambda_vector) * weighted_prod_ahpweight

In [44]:
# Create a DataFrame for ranking
ranking_df_waspass = pd.DataFrame({
    "Track Name": oldest_200_songs["track_name"],  # Song name column
    "Artist Name": oldest_200_songs["artist(s)_name"],  # Artist name column
    "WS Score entropy": weighted_sum_entweight,  # Weighted Sum scores
    "WS Score ahp": weighted_sum_ahpweight,  # Weighted Sum scores
    "WP Score entropy": weighted_prod_entweight,  # Weighted Sum scores
    "WP Score ahp": weighted_prod_ahpweight,  # Weighted Product scores
    "WASPAS Score entropy": waspas_scores_ent,  # WASPAS scores
    "WASPAS Score ahp": waspas_scores_ahp  # WASPAS scores
})

# Sort by WASPAS Score (descending order)
ranking_df_waspas_ent = ranking_df_waspass.sort_values(by="WASPAS Score entropy", ascending=False)
ranking_df_waspas_ent["WASPAS Rank"] = range(1, len(ranking_df_waspas_ent) + 1)

ranking_df_waspas_ahp = ranking_df_waspass.sort_values(by="WASPAS Score ahp", ascending=False)
ranking_df_waspas_ahp["WASPAS Rank"] = range(1, len(ranking_df_waspas_ahp) + 1)

In [45]:
print("Ranking List Using WASPAS Method:")
ranking_df_waspas_ent[["Track Name", "Artist Name", "WASPAS Score entropy", "WASPAS Rank"]].head(10)

Ranking List Using WASPAS Method:


,Track Name,Artist Name,WASPAS Score entropy,WASPAS Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.728409,1
510,Infinity,Jaymes Young,0.669217,2
452,Holly Jolly Christmas,Michael Bublï¿,0.617223,3
607,2055,Sleepy hallow,0.574606,4
721,"jealousy, jealousy",Olivia Rodrigo,0.571479,5
609,Happier Than Ever - Edit,Billie Eilish,0.554459,6
470,Driving Home for Christmas - 2019 Remaster,Chris Rea,0.524430,7
461,Wonderful Christmastime - Edited Version / Rem...,Paul McCartney,0.498214,8
439,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",0.476652,9
424,Running Up That Hill (A Deal With God),Kate Bush,0.412693,10


In [46]:
ranking_df_waspas_ent.to_excel('ranking_WASPAS_entropy.xlsx', index=False)

In [47]:
print("Ranking List Using WASPAS Method:")
ranking_df_waspas_ahp[["Track Name", "Artist Name", "WASPAS Score ahp", "WASPAS Rank"]].head(10)

Ranking List Using WASPAS Method:


,Track Name,Artist Name,WASPAS Score ahp,WASPAS Rank
84,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",0.414764,1
41,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",0.409942,2
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.402887,3
621,Lucid Dreams,Juice WRLD,0.379636,4
510,Infinity,Jaymes Young,0.342553,5
424,Running Up That Hill (A Deal With God),Kate Bush,0.326557,6
187,Circles,Post Malone,0.324695,7
526,Beggin,Mï¿½ï¿½ne,0.309449,8
516,Kiss Me More (feat. SZA),"SZA, Doja Cat",0.309135,9
166,Every Breath You Take - Remastered 2003,The Police,0.304859,10


In [48]:
ranking_df_waspas_ahp.to_excel('ranking_WASPAS_ahp.xlsx', index=False)

# TOPSIS Method

TOPSIS is based on the concept that the optimal alternative should have the shortest distance to the ideal solution and the longest distance from the negative-ideal solution. The main steps include:

- Constructing a normalized decision matrix.
- Identifying the ideal (best) and negative-ideal (worst) solutions.
- Calculating the Euclidean distances from each alternative to these solutions.
- Ranking the alternatives based on their relative closeness to the ideal solution.

TOPSIS provides a clear geometric interpretation of the decision problem.


In [49]:
df = oldest_200_songs.copy()

# Create a new column combining track_name and artist(s)_name as the alternative identifier
df['Alternative'] = df['track_name'] + " (" + df['artist(s)_name'] + ")"

# Define the criteria columns and fixed criteria types
criteria_cols = ['in_spotify_playlists', 'in_spotify_charts', 'streams',
                 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_shazam_charts']
criteria_types = ["Max", "Min", "Max", "Max", "Min", "Max", "Min"]

# Extract the matrix of criteria values
# The number of alternatives is determined by the number of rows in the dataset
X = df[criteria_cols].values

# Create column labels combining the original column names and their criteria type
col_labels = [f"{col}({ctype})" for col, ctype in zip(criteria_cols, criteria_types)]

# Build the final DataFrame (matrix) with alternatives as index and criteria as columns
matrix_df = pd.DataFrame(X, columns=col_labels, index=df['Alternative'])

matrix_df

,in_spotify_playlists(Max),in_spotify_charts(Min),streams(Max),in_apple_playlists(Max),in_apple_charts(Min),in_deezer_playlists(Max),in_shazam_charts(Min)
Alternative,,,,,,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",323.0,0.0,90598517.0,4.0,0.0,14.0,0.0
"White Christmas (Bing Crosby, John Scott Trotter & His Orchestra, Ken Darby Singers)",11940.0,0.0,395591396.0,73.0,79.0,123.0,0.0
The Christmas Song (Merry Christmas To You) - Remastered 1999 (Nat King Cole),11500.0,0.0,389771964.0,140.0,72.0,251.0,0.0
"Let It Snow! Let It Snow! Let It Snow! (Frank Sinatra, B. Swanson Quartet)",10585.0,0.0,473248298.0,126.0,108.0,406.0,0.0
A Holly Jolly Christmas - Single Version (Burl Ives),7930.0,0.0,395591396.0,108.0,120.0,73.0,0.0
...,...,...,...,...,...,...,...
Flowers (Lauren Spencer Smith),801.0,0.0,184826429.0,42.0,9.0,24.0,1.0
Every Summertime (NIKI),1211.0,2.0,290228626.0,30.0,2.0,5.0,6.0
"Se Le Ve (Arcangel, De La Ghetto, Justin Quiles, Lenny Tavï¿½ï¿½rez, Sech, Dalex, Dimelo Flow, Rich Music)",1560.0,0.0,223319934.0,72.0,0.0,0.0,0.0


1. **Normalization:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\sum_{i=1}^{n} x_{ij}^2}}
$$

In [50]:
X_values = matrix_df.values.astype(float)

# Calculate the norm (sqrt of sum of squares) for each column
norms_matrix = np.linalg.norm(X_values, axis=0)

# Normalize each value in the matrix
topsis_normalized_matrix = X_values / norms_matrix

2. **Weighted Normalized Matrix:**
$$
v_{ij} = w_j \cdot r_{ij}
$$

In [51]:
#using entropy weights
topsis_weighted_matrix_ent = topsis_normalized_matrix * wj_matrix

#using entropy weights
topsis_weighted_matrix_ahp = topsis_normalized_matrix * weights_ahp

3. **Ideal and Negative-Ideal Solutions:**
$$
v_j^+ = \max_i \{v_{ij}\} \quad \text{and} \quad v_j^- = \min_i \{v_{ij}\}
$$

In [52]:
ideal_ent = []
anti_ideal_ent = []

for j, ctype in enumerate(criteria_types):
    if ctype == 'max':
        ideal_ent.append(np.max(topsis_weighted_matrix_ent[:, j]))     # best (max)
        anti_ideal_ent.append(np.min(topsis_weighted_matrix_ent[:, j])) # worst (min)
    else:  # ctype == 'min'
        ideal_ent.append(np.min(topsis_weighted_matrix_ent[:, j]))      # best (min)
        anti_ideal_ent.append(np.max(topsis_weighted_matrix_ent[:, j])) # worst (max)

ideal_ent = np.array(ideal_ent)
anti_ideal_ent = np.array(anti_ideal_ent)

print("Ideal Solution:", ideal_ent)
print("Anti-Ideal Solution:", anti_ideal_ent)

Ideal Solution: [1.63681817e-05 0.00000000e+00 1.33534666e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Anti-Ideal Solution: [0.01557564 0.0990172  0.00976202 0.02788392 0.02686623 0.02179467
 0.16352923]


In [53]:
ideal_ahp = []
anti_ideal_ahp = []

for j, ctype in enumerate(criteria_types):
    if ctype == 'max':
        ideal_ahp.append(np.max(topsis_weighted_matrix_ahp[:, j]))     # best (max)
        anti_ideal_ahp.append(np.min(topsis_weighted_matrix_ahp[:, j])) # worst (min)
    else:  # ctype == 'min'
        ideal_ahp.append(np.min(topsis_weighted_matrix_ahp[:, j]))      # best (min)
        anti_ideal_ahp.append(np.max(topsis_weighted_matrix_ahp[:, j])) # worst (max)

ideal_ahp = np.array(ideal_ahp)
anti_ideal_ahp = np.array(anti_ideal_ahp)

print("Ideal Solution:", ideal_ahp)
print("Anti-Ideal Solution:", anti_ideal_ahp)

Ideal Solution: [3.11062792e-05 0.00000000e+00 1.36331184e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Anti-Ideal Solution: [0.02960013 0.02149092 0.09966458 0.02555947 0.01393896 0.04368709
 0.01235623]


4. **Euclidean Distances:**
$$
d_i^+ = \sqrt{\sum_{j=1}^{m} (v_{ij} - v_j^+)^2}, \quad d_i^- = \sqrt{\sum_{j=1}^{m} (v_{ij} - v_j^-)^2}
$$

In [54]:
# Distance to ideal solution
dist_ideal_ent = np.sqrt(np.sum((topsis_weighted_matrix_ent - ideal_ent)**2, axis=1))

# Distance to anti-ideal solution
dist_anti_ideal_ent = np.sqrt(np.sum((topsis_weighted_matrix_ent - anti_ideal_ent)**2, axis=1))

# Display distances
df_dist_ent = pd.DataFrame({
    'Dist_to_Ideal': dist_ideal_ent,
    'Dist_to_Anti_Ideal': dist_anti_ideal_ent
}, index=matrix_df.index)

df_dist_ent

,Dist_to_Ideal,Dist_to_Anti_Ideal
Alternative,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",0.000454,0.197030
"White Christmas (Bing Crosby, John Scott Trotter & His Orchestra, Ken Darby Singers)",0.011377,0.194899
The Christmas Song (Merry Christmas To You) - Remastered 1999 (Nat King Cole),0.013622,0.194296
"Let It Snow! Let It Snow! Let It Snow! (Frank Sinatra, B. Swanson Quartet)",0.016907,0.193800
A Holly Jolly Christmas - Single Version (Burl Ives),0.014352,0.194535
...,...,...
Flowers (Lauren Spencer Smith),0.002693,0.196373
Every Summertime (NIKI),0.003021,0.194741
"Se Le Ve (Arcangel, De La Ghetto, Justin Quiles, Lenny Tavï¿½ï¿½rez, Sech, Dalex, Dimelo Flow, Rich Music)",0.004209,0.196489


In [55]:
# Distance to ideal solution
dist_ideal_ahp = np.sqrt(np.sum((topsis_weighted_matrix_ahp - ideal_ahp)**2, axis=1))

# Distance to anti-ideal solution
dist_anti_ideal_ahp = np.sqrt(np.sum((topsis_weighted_matrix_ahp - anti_ideal_ahp)**2, axis=1))

# Display distances
df_dist_ahp = pd.DataFrame({
    'Dist_to_Ideal': dist_ideal_ahp,
    'Dist_to_Anti_Ideal': dist_anti_ideal_ahp
}, index=matrix_df.index)

df_dist_ahp

,Dist_to_Ideal,Dist_to_Anti_Ideal
Alternative,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",0.001988,0.116039
"White Christmas (Bing Crosby, John Scott Trotter & His Orchestra, Ken Darby Singers)",0.019117,0.101407
The Christmas Song (Merry Christmas To You) - Remastered 1999 (Nat King Cole),0.021951,0.098980
"Let It Snow! Let It Snow! Let It Snow! (Frank Sinatra, B. Swanson Quartet)",0.027511,0.094451
A Holly Jolly Christmas - Single Version (Burl Ives),0.017472,0.102490
...,...,...
Flowers (Lauren Spencer Smith),0.005809,0.112496
Every Summertime (NIKI),0.009161,0.109743
"Se Le Ve (Arcangel, De La Ghetto, Justin Quiles, Lenny Tavï¿½ï¿½rez, Sech, Dalex, Dimelo Flow, Rich Music)",0.007708,0.111335


5. **Relative Closeness:**
$$
C_i = \frac{d_i^-}{d_i^+ + d_i^-}
$$

In [56]:
# Compute TOPSIS score
topsis_score_ent = dist_anti_ideal_ent / (dist_ideal_ent + dist_anti_ideal_ent)

# Create a results DataFrame
df_results_ent = matrix_df.copy()
df_results_ent['TOPSIS_Score'] = topsis_score_ent
df_results_ent['Rank'] = df_results_ent['TOPSIS_Score'].rank(ascending=False)

# Sort by rank (1 = best)
df_results_ent = df_results_ent.sort_values(by='Rank')
df_results_ent

,in_spotify_playlists(Max),in_spotify_charts(Min),streams(Max),in_apple_playlists(Max),in_apple_charts(Min),in_deezer_playlists(Max),in_shazam_charts(Min),TOPSIS_Score,Rank
Alternative,,,,,,,,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",323.0,0.0,9.059852e+07,4.0,0.0,14.0,0.0,0.997701,1.0
"Thinking with My Dick (Kevin Gates, Juicy J)",1550.0,0.0,1.090916e+08,0.0,0.0,14.0,0.0,0.995475,2.0
"Hey, Mickey! (Baby Tate)",482.0,0.0,1.227637e+08,9.0,1.0,12.0,3.0,0.995109,3.0
San Lucas (Kevin Kaarl),407.0,1.0,2.448919e+08,5.0,0.0,5.0,0.0,0.993895,4.0
La Zona (Bad Bunny),1188.0,0.0,3.126229e+08,13.0,1.0,15.0,1.0,0.992866,5.0
...,...,...,...,...,...,...,...,...,...
Gasolina (Daddy Yankee),6457.0,18.0,6.577236e+08,98.0,95.0,453.0,454.0,0.512731,196.0
"Money Trees (Kendrick Lamar, Jay Rock)",26792.0,32.0,1.093606e+09,69.0,113.0,695.0,458.0,0.473798,197.0
"Danza Kuduro (Don Omar, Lucenzo)",17138.0,37.0,1.279435e+09,119.0,81.0,974.0,503.0,0.424859,198.0


In [57]:
df_results_ent.to_excel('ranking_TOPSIS_entropy.xlsx', index=False)

In [58]:
# Compute TOPSIS score
topsis_score_ahp = dist_anti_ideal_ahp / (dist_ideal_ahp + dist_anti_ideal_ahp)

# Create a results DataFrame
df_results_ahp = matrix_df.copy()
df_results_ahp['TOPSIS_Score'] = topsis_score_ahp
df_results_ahp['Rank'] = df_results_ahp['TOPSIS_Score'].rank(ascending=False)

# Sort by rank (1 = best)
df_results_ahp = df_results_ahp.sort_values(by='Rank')
df_results_ahp

,in_spotify_playlists(Max),in_spotify_charts(Min),streams(Max),in_apple_playlists(Max),in_apple_charts(Min),in_deezer_playlists(Max),in_shazam_charts(Min),TOPSIS_Score,Rank
Alternative,,,,,,,,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",323.0,0.0,9.059852e+07,4.0,0.0,14.0,0.0,0.983153,1.0
Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe (sped up 8282),472.0,2.0,1.037625e+08,0.0,0.0,6.0,0.0,0.979612,2.0
"Thinking with My Dick (Kevin Gates, Juicy J)",1550.0,0.0,1.090916e+08,0.0,0.0,14.0,0.0,0.974612,3.0
"Hey, Mickey! (Baby Tate)",482.0,0.0,1.227637e+08,9.0,1.0,12.0,3.0,0.973660,4.0
"Sigue (Ed Sheeran, J Balvin)",1370.0,0.0,1.069331e+08,46.0,8.0,60.0,0.0,0.961499,5.0
...,...,...,...,...,...,...,...,...,...
Every Breath You Take - Remastered 2003 (The Police),22439.0,19.0,1.593271e+09,211.0,74.0,929.0,129.0,0.410917,196.0
Lucid Dreams (Juice WRLD),14749.0,0.0,2.288695e+09,188.0,34.0,710.0,5.0,0.320094,197.0
Circles (Post Malone),19664.0,16.0,2.132336e+09,391.0,73.0,633.0,37.0,0.314883,198.0


In [59]:
df_results_ahp.to_excel('ranking_TOPSIS_ahp.xlsx', index=False)

# PROMETHEE Method

PROMETHEE is an outranking method that focuses on pairwise comparisons among alternatives. Key steps include:

- Constructing preference functions to quantify the degree of preference of one alternative over another.
- Calculating positive and negative outranking flows for each alternative.
- Determining net flow values to derive the final ranking.

PROMETHEE is especially useful when visualizing the preference structure and handling complex decision-making scenarios.


In [60]:
print(matrix)

[[3.23000000e+02 0.00000000e+00 9.05985170e+07 ... 0.00000000e+00
  1.40000000e+01 0.00000000e+00]
 [1.19400000e+04 0.00000000e+00 3.95591396e+08 ... 7.90000000e+01
  1.23000000e+02 0.00000000e+00]
 [1.15000000e+04 0.00000000e+00 3.89771964e+08 ... 7.20000000e+01
  2.51000000e+02 0.00000000e+00]
 ...
 [1.56000000e+03 0.00000000e+00 2.23319934e+08 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.89000000e+03 0.00000000e+00 4.27486004e+08 ... 1.10000000e+01
  2.01000000e+02 1.00000000e+00]
 [1.24030000e+04 0.00000000e+00 6.74772936e+08 ... 6.30000000e+01
  4.65000000e+02 1.10000000e+01]]


In [61]:
def normalize_matrix_promethee(matrix, criteria_types):
    """
    Normalize a matrix column-wise based on criteria types.

    For maximization criteria:
        normalized_value = (value - min) / (max - min)
    For minimization criteria:
        normalized_value = (max - value) / (max - min)
    """
    norm_matrix = np.zeros_like(matrix, dtype=float)
    n_rows, n_cols = matrix.shape

    for j in range(n_cols):
        col = matrix[:, j]
        col_min = np.min(col)
        col_max = np.max(col)
        range_val = col_max - col_min

        # Avoid division by zero (if all values are equal)
        if range_val == 0:
            norm_matrix[:, j] = 0.0  # or set to 1.0, depending on your application
            continue

        if criteria_types[j] == "Max":
            norm_matrix[:, j] = (col - col_min) / range_val
        elif criteria_types[j] == "Min":
            norm_matrix[:, j] = (col_max - col) / range_val
        else:
            raise ValueError(f"Unknown criteria type '{criteria_types[j]}' for column {j}")

    return norm_matrix

# --- Applying Normalization ---
normalized_matrix_promethee = normalize_matrix_promethee(matrix, criteria_types)
print("\nNormalized promethee Matrix:")
print(normalized_matrix_promethee)



Normalized promethee Matrix:
[[0.00990905 1.         0.01884206 ... 1.         0.01437372 1.        ]
 [0.4041333  1.         0.12896033 ... 0.70300752 0.12628337 1.        ]
 [0.38920185 1.         0.12685921 ... 0.72932331 0.25770021 1.        ]
 ...
 [0.05188679 1.         0.06676138 ... 1.         0.         1.        ]
 [0.23276096 1.         0.14047594 ... 0.95864662 0.2063655  0.9986376 ]
 [0.41984526 1.         0.22975937 ... 0.76315789 0.47741273 0.98501362]]


In [62]:
def compute_D_matrix_without_self(normalized_matrix):
    """
    Compute the D(Mi - Mi') matrix, excluding self-comparisons (i != i').

    Parameters:
        normalized_matrix (numpy.ndarray): The normalized decision matrix.

    Returns:
        list: A list of tuples (i, j, difference_vector) where i ≠ j.
    """
    num_alternatives = normalized_matrix.shape[0]  # Number of rows (alternatives)
    D_matrix = []

    for i in range(num_alternatives):
        for j in range(num_alternatives):
            if i != j:  # Exclude self-comparisons
                difference_vector = normalized_matrix[i, :] - normalized_matrix[j, :]
                D_matrix.append((i+1, j+1, difference_vector))  # Store as (Mi, Mj, vector)

    return D_matrix


D_matrix_promethee = compute_D_matrix_without_self(normalized_matrix_promethee)

print("\nD(Mi - Mi') Matrix :")
for i, j, diff in D_matrix_promethee:
    print(f"D(M{i} - M{j}): {diff}")

Streaming output truncated to the last 5000 lines.
D(M188 - M88): [ 0.45642731 -0.22727273  0.59323928  0.98577236 -0.37218045  0.61088296
  0.0027248 ]
D(M188 - M89): [ 0.48554364 -0.32727273  0.78217702  0.9898374  -0.36842105  0.75564682
  0.        ]
D(M188 - M90): [ 0.50176463 -0.32727273  0.89563892  0.97764228 -0.34962406  0.7936345
  0.0013624 ]
D(M188 - M91): [ 0.41316004 -0.11818182  0.71499527  0.96138211 -0.20300752  0.81930185
  0.01362398]
D(M188 - M92): [ 0.50363106 -0.32727273  0.91332435  0.85365854 -0.03383459  0.67453799
  0.        ]
D(M188 - M93): [ 0.41003801 -0.32727273  0.85971064  0.66463415 -0.33458647  0.637577
  0.09945504]
D(M188 - M94): [ 0.28814307 -0.32727273  0.46847697  0.62804878 -0.13157895 -0.17043121
  0.10217984]
D(M188 - M95): [ 0.44465183 -0.21818182  0.7457794   0.93292683  0.0112782   0.7238193
  0.04087193]
D(M188 - M96): [ 0.45442514 -0.32727273  0.52877336  0.97764228 -0.33458647  0.54517454
  0.00953678]
D(M188 - M97): [ 0.43647346 -0.1909

1. **Preference Function:**
$$
P(a_i, a_j) = \text{function that measures the degree to which } a_i \text{ is preferred over } a_j
$$

In [63]:
def compute_preference_matrix(D_matrix):
    """
    Convert D(Mi - Mi') matrix to Preference Matrix Pj(a, b).

    Parameters:
        D_matrix (list): List of tuples (i, j, difference_vector) from D(Mi - Mi').

    Returns:
        list: A list of tuples (i, j, preference_vector) where negative values are replaced by 0.
    """
    P_matrix = []

    for i, j, diff_vector in D_matrix:
        # Apply correction: Keep positive values, replace negatives with 0
        preference_vector = np.where(diff_vector > 0, diff_vector, 0)
        P_matrix.append((i, j, preference_vector))

    return P_matrix

P_matrix_promethee = compute_preference_matrix(D_matrix_promethee)


# Display results
print("\nPreference Matrix Pj(a, b) for Excel Data:")
for i, j, pref in P_matrix_promethee:
    print(f"P(M{i} - M{j}): {pref}")

Streaming output truncated to the last 5000 lines.
 0.        ]
P(M188 - M41): [0.25997692 0.09090909 0.85949283 0.95121951 0.08646617 0.52977413
 0.5013624 ]
P(M188 - M42): [0.44594136 0.         0.82784428 0.86585366 0.         0.5349076
 1.        ]
P(M188 - M43): [0.25359712 0.         0.73516344 0.60365854 0.04511278 0.50102669
 0.        ]
P(M188 - M44): [0.15939324 0.         0.67075579 0.92886179 0.         0.25564682
 0.        ]
P(M188 - M45): [0.31882042 0.         0.79037849 0.9898374  0.         0.52053388
 0.        ]
P(M188 - M46): [0.29190987 0.         0.64357968 0.67479675 0.04135338 0.65195072
 0.00681199]
P(M188 - M47): [0.         0.         0.5674792  0.8597561  0.05263158 0.10574949
 0.6239782 ]
P(M188 - M48): [0.         0.         0.66545635 0.74796748 0.         0.21663244
 0.00544959]
P(M188 - M49): [0.52355097 0.         0.42748331 1.         0.         0.45379877
 0.        ]
P(M188 - M50): [0.49647075 0.         0.36255269 1.         0.         0.
 0.     

In [64]:
def compute_aggregated_preference(P_matrix, weights):
    """
    Compute the aggregated preference matrix Pi(a, b).

    Parameters:
        P_matrix (list): List of tuples (i, j, preference_vector) from Pj(a, b).
        weights (numpy.ndarray): Weights for each criterion.

    Returns:
        list: A list of tuples (i, j, aggregated_preference) where values are summed.
    """
    Pi_matrix = []

    for i, j, pref_vector in P_matrix:
        aggregated_pref = np.sum(pref_vector * weights)  # Weighted sum
        Pi_matrix.append((i, j, aggregated_pref))

    return Pi_matrix


# Compute the aggregated preference matrices
Pi_matrix_promethee_ent = compute_aggregated_preference(P_matrix_promethee, wj_matrix)
Pi_matrix_promethee_ahp = compute_aggregated_preference(P_matrix_promethee, weights_ahp)


# Display results
print("\nAggregated Preference Matrix Pi(a, b) using entropy weights:")
for i, j, pi in Pi_matrix_promethee_ent:
    print(f"Pi(M{i} - M{j}): {pi:.4f}")

print("\nAggregated Preference Matrix Pi(a, b) using ahp weights:")
for i, j, pi in Pi_matrix_promethee_ahp:
    print(f"Pi(M{i} - M{j}): {pi:.4f}")

Streaming output truncated to the last 5000 lines.
Pi(M175 - M176): 0.0128
Pi(M175 - M177): 0.0398
Pi(M175 - M178): 0.0195
Pi(M175 - M179): 0.1429
Pi(M175 - M180): 0.0294
Pi(M175 - M181): 0.0598
Pi(M175 - M182): 0.0234
Pi(M175 - M183): 0.0603
Pi(M175 - M184): 0.0850
Pi(M175 - M185): 0.0691
Pi(M175 - M186): 0.0533
Pi(M175 - M187): 0.0635
Pi(M175 - M188): 0.0259
Pi(M175 - M189): 0.0683
Pi(M175 - M190): 0.0904
Pi(M175 - M191): 0.0348
Pi(M175 - M192): 0.0461
Pi(M175 - M193): 0.0310
Pi(M175 - M194): 0.1396
Pi(M175 - M195): 0.0718
Pi(M175 - M196): 0.1336
Pi(M175 - M197): 0.1194
Pi(M175 - M198): 0.1297
Pi(M175 - M199): 0.0672
Pi(M175 - M200): 0.0313
Pi(M176 - M1): 0.1462
Pi(M176 - M2): 0.0732
Pi(M176 - M3): 0.0728
Pi(M176 - M4): 0.0666
Pi(M176 - M5): 0.0872
Pi(M176 - M6): 0.1038
Pi(M176 - M7): 0.0811
Pi(M176 - M8): 0.0979
Pi(M176 - M9): 0.1226
Pi(M176 - M10): 0.0536
Pi(M176 - M11): 0.0362
Pi(M176 - M12): 0.0708
Pi(M176 - M13): 0.0933
Pi(M176 - M14): 0.0156
Pi(M176 - M15): 0.0620
Pi(M176 - M16

In [65]:
def convert_to_matrix(Pi_matrix, num_alternatives):
    """
    Convert a list of (i, j, value) tuples into a structured matrix format without labels.

    Parameters:
        Pi_matrix (list): List of tuples (i, j, aggregated_preference).
        num_alternatives (int): Number of alternatives.

    Returns:
        np.ndarray: Matrix with alternatives as rows and columns.
    """
    # Initialize a matrix with NaN values on the diagonal
    Pi_matrix_formatted = np.full((num_alternatives, num_alternatives), np.nan)

    # Fill in the values from Pi_matrix
    for i, j, value in Pi_matrix:
        Pi_matrix_formatted[i, j] = value  # Assign value to matrix

    return Pi_matrix_formatted

# Define the number of alternatives
num_alternatives_promethee = max(max(i, j) for i, j, _ in Pi_matrix_promethee_ent) + 1

# Convert both matrices
Pi_matrix_promethee_np_ent = convert_to_matrix(Pi_matrix_promethee_ent, num_alternatives_promethee)
Pi_matrix_promethee_np_ahp = convert_to_matrix(Pi_matrix_promethee_ahp, num_alternatives_promethee)


# Display results
print("\nAggregated Preference Matrix Pi(a, b) using entropy weights:")
print(Pi_matrix_promethee_np_ent)

print("\nAggregated Preference Matrix Pi(a, b) using ahp weights:")
print(Pi_matrix_promethee_np_ahp)


Aggregated Preference Matrix Pi(a, b) using entropy weights:
[[       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan 0.03027719 ... 0.0015745  0.00468482 0.02930423]
 [       nan 0.05564046        nan ... 0.03966002 0.01163816 0.00515913]
 ...
 [       nan 0.01755494 0.03027719 ...        nan 0.00468482 0.02930423]
 [       nan 0.06292965 0.04451972 ... 0.04694921        nan 0.02461941]
 [       nan 0.1201031  0.07059473 ... 0.10412266 0.05717345        nan]]

Aggregated Preference Matrix Pi(a, b) using ahp weights:
[[       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan 0.01570865 ... 0.00315606 0.00222272 0.01291698]
 [       nan 0.13373544        nan ... 0.09890857 0.02126142 0.00038982]
 ...
 [       nan 0.03798293 0.01570865 ...        nan 0.00222272 0.01291698]
 [       nan 0.14358587 0.04459778 ... 0.108759          nan 0.01069426]
 [       nan 0.27606063 0.14550669 ... 0.24123377 0.13247477      

In [66]:
valid_rows_ent = ~np.all(np.isnan(Pi_matrix_promethee_np_ent), axis=1)
valid_cols_ent = ~np.all(np.isnan(Pi_matrix_promethee_np_ent), axis=0)
Pi_clean_ent = Pi_matrix_promethee_np_ent[valid_rows_ent][:, valid_cols_ent]

# Step 2: Replace diagonal NaN values with 0.
# Here we assume that only the diagonal may be NaN after cleaning.
np.fill_diagonal(Pi_clean_ent, 0)

# Now, the cleaned matrix is ready for further computation.
print("Cleaned Aggregated Preference Matrix using entropy weights:")
print(Pi_clean_ent)

Cleaned Aggregated Preference Matrix using entropy weights:
[[0.         0.03027719 0.0275944  ... 0.0015745  0.00468482 0.02930423]
 [0.05564046 0.         0.00106375 ... 0.03966002 0.01163816 0.00515913]
 [0.08153754 0.02964361 0.         ... 0.0655571  0.019664   0.00515913]
 ...
 [0.01755494 0.03027719 0.0275944  ... 0.         0.00468482 0.02930423]
 [0.06292965 0.04451972 0.02396569 ... 0.04694921 0.         0.02461941]
 [0.1201031  0.07059473 0.04201486 ... 0.10412266 0.05717345 0.        ]]


In [67]:
valid_rows_ahp = ~np.all(np.isnan(Pi_matrix_promethee_np_ahp), axis=1)
valid_cols_ahp = ~np.all(np.isnan(Pi_matrix_promethee_np_ahp), axis=0)
Pi_clean_ahp = Pi_matrix_promethee_np_ahp[valid_rows_ahp][:, valid_cols_ahp]

# Step 2: Replace diagonal NaN values with 0.
# Here we assume that only the diagonal may be NaN after cleaning.
np.fill_diagonal(Pi_clean_ahp, 0)

# Now, the cleaned matrix is ready for further computation.
print("Cleaned Aggregated Preference Matrix using ahp weights:")
print(Pi_clean_ahp)

Cleaned Aggregated Preference Matrix using ahp weights:
[[0.         0.01570865 0.01431675 ... 0.00315606 0.00222272 0.01291698]
 [0.13373544 0.         0.00277429 ... 0.09890857 0.02126142 0.00038982]
 [0.17133447 0.04176523 0.         ... 0.13650761 0.03377804 0.00038982]
 ...
 [0.03798293 0.01570865 0.01431675 ... 0.         0.00222272 0.01291698]
 [0.14358587 0.04459778 0.01812346 ... 0.108759   0.         0.01069426]
 [0.27606063 0.14550669 0.10651575 ... 0.24123377 0.13247477 0.        ]]


### PROMETHEE II

2. **Positive and Negative Flows:**
$$
\phi_i^+ = \frac{1}{n - 1} \sum_{j \neq i} P(a_i, a_j), \quad \phi_i^- = \frac{1}{n - 1} \sum_{j \neq i} P(a_j, a_i)
$$

In [68]:
num_alternatives_ent = Pi_clean_ent.shape[0]
denom_ent = num_alternatives_ent - 1

phi_plus_ent = np.sum(Pi_clean_ent, axis=1) / denom_ent

phi_minus_ent = np.sum(Pi_clean_ent, axis=0) / denom_ent

print("\nPhi+ (row averages):")
print(phi_plus_ent)

print("\nPhi- (column averages):")
print(phi_minus_ent)


Phi+ (row averages):
[0.06084743 0.06617621 0.08290025 0.08876977 0.05826899 0.06397509
 0.0670795  0.07101871 0.05625139 0.08529147 0.07710964 0.07724731
 0.07505062 0.10774631 0.06282809 0.10696622 0.0645475  0.06188299
 0.12473921 0.06080793 0.17097652 0.05866343 0.12390013 0.16017188
 0.0639268  0.08037326 0.06161718 0.05845397 0.12901171 0.13401585
 0.07716699 0.04276233 0.05349888 0.05736276 0.09600976 0.13291517
 0.03179244 0.07295742 0.04325938 0.14013983 0.02837472 0.05393275
 0.0908154  0.11692275 0.08224907 0.05900737 0.11380019 0.14901539
 0.08481647 0.13045723 0.07860254 0.09687102 0.06100046 0.06536013
 0.07324693 0.07527527 0.03551682 0.07545842 0.04939467 0.04883301
 0.06427329 0.11025741 0.12418041 0.07679351 0.13385481 0.04573523
 0.05763955 0.06153838 0.08100176 0.06777392 0.06127326 0.06201398
 0.05840657 0.08174018 0.05980715 0.02946363 0.0593752  0.03885364
 0.042804   0.04372292 0.06571112 0.10392581 0.06782299 0.03389055
 0.1685098  0.09109599 0.04935358 0.0662

In [69]:
num_alternatives_ahp = Pi_clean_ahp.shape[0]
denom_ahp = num_alternatives_ahp - 1

phi_plus_ahp = np.sum(Pi_clean_ahp, axis=1) / denom_ahp

phi_minus_ahp = np.sum(Pi_clean_ahp, axis=0) / denom_ahp

print("\nPhi+ (row averages):")
print(phi_plus_ahp)

print("\nPhi- (column averages):")
print(phi_minus_ahp)


Phi+ (row averages):
[0.01690479 0.05031801 0.07298269 0.09529868 0.0351499  0.02370414
 0.03889122 0.03899449 0.01825021 0.06558575 0.07372601 0.0668489
 0.05425457 0.22439326 0.03133406 0.10288498 0.05582854 0.02383546
 0.16616231 0.02731873 0.38937986 0.02234933 0.13443974 0.25112428
 0.02618421 0.09648475 0.01989096 0.01624035 0.23143718 0.18482494
 0.05876616 0.02161869 0.09693128 0.02023744 0.20640576 0.31645342
 0.03397067 0.15165805 0.02599773 0.23198987 0.04840516 0.05022922
 0.09615172 0.15534903 0.06679897 0.08773467 0.26419542 0.23673044
 0.17886295 0.29451815 0.20899902 0.12279032 0.01725881 0.02820595
 0.06155045 0.19321372 0.06416901 0.08203775 0.03123071 0.0342652
 0.08789308 0.30278792 0.15013058 0.10984296 0.32788265 0.03580145
 0.05972513 0.02273928 0.14540085 0.04456469 0.01896657 0.02306142
 0.02532757 0.13742335 0.01681577 0.02350947 0.03870581 0.16170861
 0.03034369 0.06026663 0.0946259  0.20045225 0.07194831 0.08167883
 0.42315442 0.11259652 0.12524184 0.094399

3. **Net Flow:**
$$
\phi_i = \phi_i^+ - \phi_i^-
$$

In [70]:
Phi_ent = phi_plus_ent - phi_minus_ent

df_promethee_II_ent = pd.DataFrame({
    'Alternative': matrix_df.index,
    'Phi Entropy': Phi_ent,
}, index=matrix_df.index)

df_promethee_II_ent = df_promethee_II_ent.sort_values(by='Phi Entropy', ascending=False).reset_index(drop=True)
df_promethee_II_ent['Rank'] = df_promethee_II_ent.index + 1  # Rank starting at 1

df_promethee_II_ent

,Alternative,Phi Entropy,Rank
0,Lucid Dreams (Juice WRLD),0.159696,1
1,Running Up That Hill (A Deal With God) (Kate B...,0.153463,2
2,"Kiss Me More (feat. SZA) (SZA, Doja Cat)",0.148273,3
3,"STAY (with Justin Bieber) (Justin Bieber, The ...",0.135114,4
4,The Business (Tiï¿½ï¿),0.134224,5
...,...,...,...
195,Die For You (The Weeknd),-0.223837,196
196,"See You Again (Tyler, The Creator, Kali Uchis)",-0.231994,197
197,Radio (Lana Del Rey),-0.270283,198
198,Bloody Mary (Lady Gaga),-0.306863,199


In [71]:
df_promethee_II_ent.to_excel('ranking_PROMETHEE_entropy.xlsx', index=False)

In [72]:
Phi_ahp = phi_plus_ahp - phi_minus_ahp

df_promethee_II_ahp = pd.DataFrame({
    'Alternative': matrix_df.index,
    'Phi AHP': Phi_ahp,
}, index=matrix_df.index)

df_promethee_II_ahp = df_promethee_II_ahp.sort_values(by='Phi AHP', ascending=False).reset_index(drop=True)
df_promethee_II_ahp['Rank'] = df_promethee_II_ahp.index + 1  # Rank starting at 1

df_promethee_II_ahp

,Alternative,Phi AHP,Rank
0,Sunflower - Spider-Man: Into the Spider-Verse ...,0.542659,1
1,"STAY (with Justin Bieber) (Justin Bieber, The ...",0.526990,2
2,Lucid Dreams (Juice WRLD),0.414855,3
3,Circles (Post Malone),0.411604,4
4,Every Breath You Take - Remastered 2003 (The P...,0.368258,5
...,...,...,...
195,Por las Noches (Peso Pluma),-0.177261,196
196,"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",-0.179697,197
197,Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½...,-0.180435,198
198,295 (Sidhu Moose Wala),-0.191756,199


In [73]:
df_promethee_II_ahp.to_excel('ranking_PROMETHEE_ahp.xlsx', index=False)

### PROMETHEE I

In [74]:
print("Cleaned Aggregated Preference Matrix using Entropy weights:")
print(Pi_clean_ent)

phi_plus_I_ent = np.sum(Pi_clean_ent, axis=1)

phi_minus_I_ent = np.sum(Pi_clean_ent, axis=0)

print("\nPhi+ (row averages):")
print(phi_plus_I_ent)

print("\nPhi- (column averages):")
print(phi_minus_I_ent)

Cleaned Aggregated Preference Matrix using Entropy weights:
[[0.         0.03027719 0.0275944  ... 0.0015745  0.00468482 0.02930423]
 [0.05564046 0.         0.00106375 ... 0.03966002 0.01163816 0.00515913]
 [0.08153754 0.02964361 0.         ... 0.0655571  0.019664   0.00515913]
 ...
 [0.01755494 0.03027719 0.0275944  ... 0.         0.00468482 0.02930423]
 [0.06292965 0.04451972 0.02396569 ... 0.04694921 0.         0.02461941]
 [0.1201031  0.07059473 0.04201486 ... 0.10412266 0.05717345 0.        ]]

Phi+ (row averages):
[12.10863853 13.16906626 16.49714975 17.66518437 11.59552974 12.73104317
 13.34882056 14.13272388 11.19402702 16.9730034  15.34481784 15.37221566
 14.93507373 21.44151498 12.5027907  21.28627735 12.84495309 12.31471537
 24.82310201 12.10077712 34.02432701 11.67402194 24.65612513 31.87420353
 12.72143389 15.99427882 12.26181868 11.63234042 25.67332984 26.66915473
 15.35623164  8.50970287 10.64627725 11.41518996 19.10594227 26.45011884
  6.32669577 14.51852591  8.60861737

In [75]:
print("Cleaned Aggregated Preference Matrix using AHP weights:")
print(Pi_clean_ahp)

phi_plus_I_ahp = np.sum(Pi_clean_ahp, axis=1)

phi_minus_I_ahp = np.sum(Pi_clean_ahp, axis=0)

print("\nPhi+ (row averages):")
print(phi_plus_I_ahp)

print("\nPhi- (column averages):")
print(phi_minus_I_ahp)

Cleaned Aggregated Preference Matrix using AHP weights:
[[0.         0.01570865 0.01431675 ... 0.00315606 0.00222272 0.01291698]
 [0.13373544 0.         0.00277429 ... 0.09890857 0.02126142 0.00038982]
 [0.17133447 0.04176523 0.         ... 0.13650761 0.03377804 0.00038982]
 ...
 [0.03798293 0.01570865 0.01431675 ... 0.         0.00222272 0.01291698]
 [0.14358587 0.04459778 0.01812346 ... 0.108759   0.         0.01069426]
 [0.27606063 0.14550669 0.10651575 ... 0.24123377 0.13247477 0.        ]]

Phi+ (row averages):
[  3.36405308  10.01328373  14.52355485  18.96443691   6.99483093
   4.71712392   7.73935338   7.75990334   3.63179117  13.05156353
  14.67147573  13.30293094  10.79666021  44.65425855   6.2354789
  20.47411166  11.10987921   4.7432559   33.06629996   5.43642629
  77.48659125   4.44751744  26.75350925  49.97373232   5.21065726
  19.20046436   3.95830193   3.23183009  46.05599792  36.7801627
  11.69446545   4.30211995  19.28932416   4.02725019  41.0747464
  62.97423007   6.7

In [76]:
df_prom_I_ent = pd.DataFrame({
    'Alternative': matrix_df.index,
    'Phi_plus_entropy': phi_plus_I_ent,
    'Phi_minus_entropy': phi_minus_I_ent
})

df_prom_I_ent

,Alternative,Phi_plus_entropy,Phi_minus_entropy
0,"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",12.108639,13.035482
1,"White Christmas (Bing Crosby, John Scott Trott...",13.169066,9.023255
2,The Christmas Song (Merry Christmas To You) - ...,16.497150,6.635366
3,Let It Snow! Let It Snow! Let It Snow! (Frank ...,17.665184,7.746414
4,A Holly Jolly Christmas - Single Version (Burl...,11.595530,12.178047
...,...,...,...
195,Flowers (Lauren Spencer Smith),11.897849,11.453284
196,Every Summertime (NIKI),11.475942,12.217943
197,"Se Le Ve (Arcangel, De La Ghetto, Justin Quile...",13.032058,10.762813
198,OUT OUT (feat. Charli XCX & Saweetie) (Charli ...,16.205072,5.482950


In [77]:
df_prom_I_ahp = pd.DataFrame({
    'Alternative': matrix_df.index,
    'Phi_plus_AHP': phi_plus_I_ahp,
    'Phi_minus_AHP': phi_minus_I_ahp
})

df_prom_I_ahp

,Alternative,Phi_plus_AHP,Phi_minus_AHP
0,"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",3.364053,39.123678
1,"White Christmas (Bing Crosby, John Scott Trott...",10.013284,22.167551
2,The Christmas Song (Merry Christmas To You) - ...,14.523555,18.879634
3,Let It Snow! Let It Snow! Let It Snow! (Frank ...,18.964437,16.360875
4,A Holly Jolly Christmas - Single Version (Burl...,6.994831,25.201518
...,...,...,...
195,Flowers (Lauren Spencer Smith),3.588194,34.558446
196,Every Summertime (NIKI),3.846703,32.340544
197,"Se Le Ve (Arcangel, De La Ghetto, Justin Quile...",4.427276,33.221528
198,OUT OUT (feat. Charli XCX & Saweetie) (Charli ...,11.414583,18.901578


In [78]:
def compare_promethee_i(phi_plus_a, phi_minus_a, phi_plus_b, phi_minus_b):
    """
    Returns the relationship of a relative to b in PROMETHEE I:
    - 'P' if a strictly outranks b (aPb)
    - 'I' if a is indifferent to b (aIb)
    - 'R' if a is incomparable to b (aRb)
    """
    # Unpack for easier reading
    pa, ma = phi_plus_a, phi_minus_a
    pb, mb = phi_plus_b, phi_minus_b

    # Strict Preference Conditions (a P b)
    # 1) pa > pb and ma < mb
    # 2) pa > pb and ma = mb
    # 3) pa = pb and ma < mb
    if (pa > pb and ma < mb) or (pa > pb and ma == mb) or (pa == pb and ma < mb):
        return "P"

    # Indifference Condition (a I b)
    # pa = pb and ma = mb
    if pa == pb and ma == mb:
        return "I"

    # Incomparability Conditions (a R b)
    # 1) pa > pb and ma > mb
    # 2) pa < pb and ma < mb
    return "R"

In [79]:
relationships_ent = []
for i in range(len(df)):
    for j in range(len(df)):
        if i == j:
            # an alternative compared to itself is not relevant here
            continue

        a = df_prom_I_ent.loc[i, "Alternative"]
        b = df_prom_I_ent.loc[j, "Alternative"]
        pa, ma = df_prom_I_ent.loc[i, "Phi_plus_entropy"], df_prom_I_ent.loc[i, "Phi_minus_entropy"]
        pb, mb = df_prom_I_ent.loc[j, "Phi_plus_entropy"], df_prom_I_ent.loc[j, "Phi_minus_entropy"]

        rel = compare_promethee_i(pa, ma, pb, mb)

        relationships_ent.append({
            "Alternative_A": a,
            "Alternative_B": b,
            "Relationship": rel,
        })

rel_df_ent = pd.DataFrame(relationships_ent)

In [80]:
relationships_ahp = []
for i in range(len(df)):
    for j in range(len(df)):
        if i == j:
            # an alternative compared to itself is not relevant here
            continue

        a = df_prom_I_ahp.loc[i, "Alternative"]
        b = df_prom_I_ahp.loc[j, "Alternative"]
        pa, ma = df_prom_I_ahp.loc[i, "Phi_plus_AHP"], df_prom_I_ahp.loc[i, "Phi_minus_AHP"]
        pb, mb = df_prom_I_ahp.loc[j, "Phi_plus_AHP"], df_prom_I_ahp.loc[j, "Phi_minus_AHP"]

        rel = compare_promethee_i(pa, ma, pb, mb)

        relationships_ahp.append({
            "Alternative_A": a,
            "Alternative_B": b,
            "Relationship": rel,
        })

rel_df_ahp = pd.DataFrame(relationships_ahp)

In [81]:
alternatives_ent = df_prom_I_ent['Alternative']
# Initialize adjacency sets
graph_ent = { alt: set() for alt in alternatives_ent }

for row in rel_df_ent.itertuples():
    a = row.Alternative_A
    b = row.Alternative_B
    rel = row.Relationship

    if rel == "P":
        # a P b means: a strictly outranks b
        graph_ent[a].add(b)

# For convenience, also track "incoming edges" (who outranks me?)
incoming_ent = { alt: set() for alt in alternatives_ent }
for a in graph_ent:
    for b in graph_ent[a]:
        incoming_ent[b].add(a)

print("\nStrict Preference Graph (who each alternative outranks):")
for alt in graph_ent:
    print(f"{alt} -> {graph_ent[alt]}")


Strict Preference Graph (who each alternative outranks):
Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth) -> {'Lovers Rock (TV Girl)', 'See You Again (Tyler, The Creator, Kali Uchis)', 'Yo Voy (feat. Daddy Yankee) (Zion & Lennox)', 'After Hours (The Weeknd)', 'Miï¿½ï¿½n (Tini, Maria Becerra)', 'Permission to Dance (BTS)', 'All Of The Girls You Loved Before (Taylor Swift)', 'La Santa (Daddy Yankee, Bad Bunny)', 'I Was Never There (The Weeknd, Gesaffelstein)', 'After Dark (Mr.Kitty)', 'Formula (Labrinth)', 'ýýýýýýýýýýýýýýýýýýýýý (Fujii Kaze)', 'happier (Olivia Rodrigo)', 'Cï¿½ï¿½ (Rauw Alejandro)', 'Cruel Summer (Taylor Swift)', 'Die For You (The Weeknd)', 'cardigan (Taylor Swift)', 'Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe (sped up 8282)', 'Sex, Drugs, Etc. (Beach Weather)', 'Stargirl Interlude (The Weeknd, Lana Del Rey)', 'Reminder (The Weeknd)', 'Shut up My Moms Calling (Hotel Ugly)', 'Mary On A Cross (Ghost)', 'Es un Secreto (Plan B)', 'All For Us - from the HBO Original Serie

In [82]:
alternatives_ahp = df_prom_I_ahp['Alternative']
# Initialize adjacency sets
graph_ahp = { alt: set() for alt in alternatives_ahp }

for row in rel_df_ahp.itertuples():
    a = row.Alternative_A
    b = row.Alternative_B
    rel = row.Relationship

    if rel == "P":
        # a P b means: a strictly outranks b
        graph_ahp[a].add(b)

# For convenience, also track "incoming edges" (who outranks me?)
incoming_ahp = { alt: set() for alt in alternatives_ahp }
for a in graph_ahp:
    for b in graph_ahp[a]:
        incoming_ahp[b].add(a)

print("\nStrict Preference Graph (who each alternative outranks):")
for alt in graph_ahp:
    print(f"{alt} -> {graph_ahp[alt]}")


Strict Preference Graph (who each alternative outranks):
Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth) -> {'Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe (sped up 8282)', '295 (Sidhu Moose Wala)', 'Still With You (Jung Kook)'}
White Christmas (Bing Crosby, John Scott Trotter & His Orchestra, Ken Darby Singers) -> {'Lovers Rock (TV Girl)', 'Heart To Heart (Mac DeMarco)', 'Yo Voy (feat. Daddy Yankee) (Zion & Lennox)', 'Miï¿½ï¿½n (Tini, Maria Becerra)', 'Permission to Dance (BTS)', 'All Of The Girls You Loved Before (Taylor Swift)', 'Cï¿½ï¿½ (Rauw Alejandro)', 'After Dark (Mr.Kitty)', 'Notion (The Rare Occasions)', 'Formula (Labrinth)', 'ýýýýýýýýýýýýýýýýýýýýý (Fujii Kaze)', 'A Holly Jolly Christmas - Single Version (Burl Ives)', 'Driving Home for Christmas - 2019 Remaster (Chris Rea)', 'Sky (Playboi Carti)', 'Demasiadas Mujeres (C. Tangana)', 'Jordan (Ryan Castro)', 'Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)', 'Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe (sped up 8282)', "M

In [83]:
graph_copy_ent = copy.deepcopy(graph_ent)
incoming_copy_ent = copy.deepcopy(incoming_ent)
all_alts = set(alternatives_ent)

layers_ent = []  # list of lists, each sub-list is a "layer" in the partial order

while all_alts:
    # 1) Find all nodes with no incoming edges
    no_incoming = [alt for alt in all_alts if len(incoming_copy_ent[alt]) == 0]

    if not no_incoming:
        # If there's a cycle or only incomparabilities preventing further layering,
        # we break. The remaining ones are either a cycle or can't be ranked strictly.
        break

    # 2) This set forms the next "layer"
    layers_ent.append(no_incoming)

    # 3) Remove them from the graph
    for alt in no_incoming:
        # Remove alt from the graph
        all_alts.remove(alt)
        # Remove alt's edges
        for out_alt in graph_copy_ent[alt]:
            incoming_copy_ent[out_alt].remove(alt)
        graph_copy_ent[alt].clear()

print("\nPartial Ranking Layers (PROMETHEE I):")
for i, layer in enumerate(layers_ent, start=1):
    print(f"Level {i}: {layer}")


Partial Ranking Layers (PROMETHEE I):
Level 1: ['Lucid Dreams (Juice WRLD)', 'Circles (Post Malone)', 'Running Up That Hill (A Deal With God) (Kate Bush)', 'STAY (with Justin Bieber) (Justin Bieber, The Kid Laroi)']
Level 2: ['Cool for the Summer (Demi Lovato)', 'The Business (Tiï¿½ï¿)', 'Sunflower - Spider-Man: Into the Spider-Verse (Post Malone, Swae Lee)', 'Kiss Me More (feat. SZA) (SZA, Doja Cat)', 'Every Breath You Take - Remastered 2003 (The Police)']
Level 3: ["Do They Know It's Christmas? - 1984 Version (Band Aid)", 'Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit (Riton, Nightcrawlers, Mufasa & Hypeman, Dopamine)', 'Beggin (Mï¿½ï¿½ne)', 'Lost (Frank Ocean)']
Level 4: ['Astronaut In The Ocean (Masked Wolf)', 'Talking To The Moon (Bruno Mars)', 'Adore You (Harry Styles)', 'Iris (The Goo Goo Dolls)', "It's Beginning To Look A Lot Like Christmas (Michael Bublï¿)"]
Level 5: ['Happy Xmas (War Is Over) (John Lennon, The Harlem Community Choir, The Plastic Ono Band, Yoko Ono)', 'S

In [100]:
ranking_list_ent = []
for level, layer in enumerate(layers_ent, start=1):
    for song in layer:
        ranking_list_ent.append({
            'Song': song,
            'PROMETHEE_I_Level': level
        })

# Convert to DataFrame
df_promethee_ranking_ent = pd.DataFrame(ranking_list_ent)

# Optionally, sort the DataFrame by the ranking level
df_promethee_ranking_ent = df_promethee_ranking_ent.sort_values('PROMETHEE_I_Level').reset_index(drop=True)

print("PROMETHEE I Ranking Levels:")
df_promethee_ranking_ent

PROMETHEE I Ranking Levels:


,Song,PROMETHEE_I_Level
0,Lucid Dreams (Juice WRLD),1
1,Circles (Post Malone),1
2,Running Up That Hill (A Deal With God) (Kate B...,1
3,"STAY (with Justin Bieber) (Justin Bieber, The ...",1
4,Cool for the Summer (Demi Lovato),2
5,The Business (Tiï¿½ï¿),2
6,Sunflower - Spider-Man: Into the Spider-Verse ...,2
7,"Kiss Me More (feat. SZA) (SZA, Doja Cat)",2
8,Every Breath You Take - Remastered 2003 (The P...,2
9,Do They Know It's Christmas? - 1984 Version (B...,3


In [84]:
graph_copy_ahp = copy.deepcopy(graph_ahp)
incoming_copy_ahp = copy.deepcopy(incoming_ahp)
all_alts = set(alternatives_ahp)

layers_ahp = []  # list of lists, each sub-list is a "layer" in the partial order

while all_alts:
    # 1) Find all nodes with no incoming edges
    no_incoming = [alt for alt in all_alts if len(incoming_copy_ahp[alt]) == 0]

    if not no_incoming:
        # If there's a cycle or only incomparabilities preventing further layering,
        # we break. The remaining ones are either a cycle or can't be ranked strictly.
        break

    # 2) This set forms the next "layer"
    layers_ahp.append(no_incoming)

    # 3) Remove them from the graph
    for alt in no_incoming:
        # Remove alt from the graph
        all_alts.remove(alt)
        # Remove alt's edges
        for out_alt in graph_copy_ahp[alt]:
            incoming_copy_ahp[out_alt].remove(alt)
        graph_copy_ahp[alt].clear()

print("\nPartial Ranking Layers (PROMETHEE I):")
for i, layer in enumerate(layers_ahp, start=1):
    print(f"Level {i}: {layer}")


Partial Ranking Layers (PROMETHEE I):
Level 1: ['Lucid Dreams (Juice WRLD)', 'Circles (Post Malone)', 'Sunflower - Spider-Man: Into the Spider-Verse (Post Malone, Swae Lee)', 'STAY (with Justin Bieber) (Justin Bieber, The Kid Laroi)']
Level 2: ['Levitating (feat. DaBaby) (Dua Lipa, DaBaby)', 'Every Breath You Take - Remastered 2003 (The Police)']
Level 3: ['Beggin (Mï¿½ï¿½ne)', 'Kiss Me More (feat. SZA) (SZA, Doja Cat)', 'The Night We Met (Lord Huron)']
Level 4: ['Adore You (Harry Styles)', 'Iris (The Goo Goo Dolls)', 'Call Out My Name (The Weeknd)', 'No Role Modelz (J. Cole)', 'Danza Kuduro (Don Omar, Lucenzo)']
Level 5: ['Let Me Down Slowly (Alec Benjamin)', 'Running Up That Hill (A Deal With God) (Kate Bush)', 'When I Was Your Man (Bruno Mars)']
Level 6: ['Talking To The Moon (Bruno Mars)', 'The Business (Tiï¿½ï¿)', 'Dakiti (Bad Bunny, Jhay Cortez)', 'Before You Go (Lewis Capaldi)', 'Have You Ever Seen The Rain? (Creedence Clearwater Revival)', 'Lost (Frank Ocean)']
Level 7: ['Astr

In [101]:
ranking_list_ahp = []
for level, layer in enumerate(layers_ahp, start=1):
    for song in layer:
        ranking_list_ahp.append({
            'Song': song,
            'PROMETHEE_I_Level': level
        })

# Convert to DataFrame
df_promethee_ranking_ahp = pd.DataFrame(ranking_list_ahp)

# Optionally, sort the DataFrame by the ranking level
df_promethee_ranking_ahp = df_promethee_ranking_ahp.sort_values('PROMETHEE_I_Level').reset_index(drop=True)

print("PROMETHEE I Ranking Levels:")
df_promethee_ranking_ahp

PROMETHEE I Ranking Levels:


,Song,PROMETHEE_I_Level
0,Lucid Dreams (Juice WRLD),1
1,Circles (Post Malone),1
2,Sunflower - Spider-Man: Into the Spider-Verse ...,1
3,"STAY (with Justin Bieber) (Justin Bieber, The ...",1
4,"Levitating (feat. DaBaby) (Dua Lipa, DaBaby)",2
...,...,...
86,"Don't Be Shy (Tiï¿½ï¿½sto, Kar)",24
87,Dandelions (Ruth B.),24
88,Let It Snow! Let It Snow! Let It Snow! (Dean M...,24
89,"Volvï¿ (Aventura, Bad Bunny)",24


# Comparison

1. Weighted sum

In [128]:
ranking_df_entropy[["Track Name", "Artist Name", "Entropy Weighted Score", "Entropy Rank"]].head(5)

,Track Name,Artist Name,Entropy Weighted Score,Entropy Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.797896,1
510,Infinity,Jaymes Young,0.762236,2
424,Running Up That Hill (A Deal With God),Kate Bush,0.749845,3
452,Holly Jolly Christmas,Michael Bublï¿,0.747787,4
721,"jealousy, jealousy",Olivia Rodrigo,0.717064,5


In [127]:
ranking_df_ahp[["Track Name", "Artist Name", "AHP Weighted Score", "AHP Rank"]].head(5)

,Track Name,Artist Name,AHP Weighted Score,AHP Rank
41,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",0.795348,1
84,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",0.779891,2
621,Lucid Dreams,Juice WRLD,0.665969,3
187,Circles,Post Malone,0.626742,4
166,Every Breath You Take - Remastered 2003,The Police,0.589675,5


2. Weighted product

In [117]:
ranking_df_entropy_prod[["Track Name", "Artist Name", "Entropy Weighted Score", "Entropy Rank"]].head(5)

,Track Name,Artist Name,Entropy Weighted Score,Entropy Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.658921,1
510,Infinity,Jaymes Young,0.576198,2
452,Holly Jolly Christmas,Michael Bublï¿,0.486658,3
607,2055,Sleepy hallow,0.434419,4
721,"jealousy, jealousy",Olivia Rodrigo,0.425895,5


In [126]:
ranking_df_ahp_prod[["Track Name", "Artist Name", "AHP Weighted Score", "AHP Rank"]].head(5)

,Track Name,Artist Name,AHP Weighted Score,AHP Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.365738,1
510,Infinity,Jaymes Young,0.302333,2
452,Holly Jolly Christmas,Michael Bublï¿,0.201587,3
721,"jealousy, jealousy",Olivia Rodrigo,0.162979,4
424,Running Up That Hill (A Deal With God),Kate Bush,0.149470,5


3. WASPAS

In [118]:
ranking_df_waspas_ent[["Track Name", "Artist Name", "WASPAS Score entropy", "WASPAS Rank"]].head(5)

,Track Name,Artist Name,WASPAS Score entropy,WASPAS Rank
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.728409,1
510,Infinity,Jaymes Young,0.669217,2
452,Holly Jolly Christmas,Michael Bublï¿,0.617223,3
607,2055,Sleepy hallow,0.574606,4
721,"jealousy, jealousy",Olivia Rodrigo,0.571479,5


In [125]:
ranking_df_waspas_ahp[["Track Name", "Artist Name", "WASPAS Score ahp", "WASPAS Rank"]].head(5)

,Track Name,Artist Name,WASPAS Score ahp,WASPAS Rank
84,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",0.414764,1
41,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",0.409942,2
445,It's Beginning To Look A Lot Like Christmas,Michael Bublï¿,0.402887,3
621,Lucid Dreams,Juice WRLD,0.379636,4
510,Infinity,Jaymes Young,0.342553,5


4. TOPSIS

In [119]:
df_results_ent[["TOPSIS_Score", "Rank"]].head(5)

,TOPSIS_Score,Rank
Alternative,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",0.997701,1.0
"Thinking with My Dick (Kevin Gates, Juicy J)",0.995475,2.0
"Hey, Mickey! (Baby Tate)",0.995109,3.0
San Lucas (Kevin Kaarl),0.993895,4.0
La Zona (Bad Bunny),0.992866,5.0


In [124]:
df_results_ahp[["TOPSIS_Score", "Rank"]].head(5)

,TOPSIS_Score,Rank
Alternative,,
"Agudo Mï¿½ï¿½gi (Styrx, utku INC, Thezth)",0.983153,1.0
Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe (sped up 8282),0.979612,2.0
"Thinking with My Dick (Kevin Gates, Juicy J)",0.974612,3.0
"Hey, Mickey! (Baby Tate)",0.973660,4.0
"Sigue (Ed Sheeran, J Balvin)",0.961499,5.0


5. PROMETHEE II

In [120]:
df_promethee_II_ent.head(5)

,Alternative,Phi Entropy,Rank
0,Lucid Dreams (Juice WRLD),0.159696,1
1,Running Up That Hill (A Deal With God) (Kate B...,0.153463,2
2,"Kiss Me More (feat. SZA) (SZA, Doja Cat)",0.148273,3
3,"STAY (with Justin Bieber) (Justin Bieber, The ...",0.135114,4
4,The Business (Tiï¿½ï¿),0.134224,5


In [123]:
df_promethee_II_ahp.head(5)

,Alternative,Phi AHP,Rank
0,Sunflower - Spider-Man: Into the Spider-Verse ...,0.542659,1
1,"STAY (with Justin Bieber) (Justin Bieber, The ...",0.526990,2
2,Lucid Dreams (Juice WRLD),0.414855,3
3,Circles (Post Malone),0.411604,4
4,Every Breath You Take - Remastered 2003 (The P...,0.368258,5


6. PROMETHEE I

In [121]:
df_promethee_ranking_ent.head(5)

,Song,PROMETHEE_I_Level
0,Lucid Dreams (Juice WRLD),1
1,Circles (Post Malone),1
2,Running Up That Hill (A Deal With God) (Kate B...,1
3,"STAY (with Justin Bieber) (Justin Bieber, The ...",1
4,Cool for the Summer (Demi Lovato),2


In [122]:
df_promethee_ranking_ahp.head(5)

,Song,PROMETHEE_I_Level
0,Lucid Dreams (Juice WRLD),1
1,Circles (Post Malone),1
2,Sunflower - Spider-Man: Into the Spider-Verse ...,1
3,"STAY (with Justin Bieber) (Justin Bieber, The ...",1
4,"Levitating (feat. DaBaby) (Dua Lipa, DaBaby)",2
